In [ ]:

import os 
os.environ["TOKENIZERS_PARALLELISM"] = "false"
from transformers import AutoTokenizer, AutoModelForCausalLM
model_list = [
    "mistralai/Mistral-7B-Instruct-v0.3",
    "mistralai/Ministral-8B-Instruct-2410",
    "mistralai/Mistral-Nemo-Instruct-2407",
    "mistralai/Mistral-Small-Instruct-2409",
    "mistralai/Mathstral-7B-v0.1",
    
    "Qwen/Qwen2.5-32B-Instruct",
    "Qwen/Qwen2.5-14B-Instruct",
    "Qwen/Qwen2.5-7B-Instruct",
    "Qwen/Qwen2.5-3B-Instruct",
    "Qwen/Qwen2.5-1.5B-Instruct",
    "Qwen/Qwen2.5-Math-1.5B-Instruct",
    "Qwen/Qwen2.5-Math-7B-Instruct",
    "Qwen/QwQ-32B-Preview",
    
    "microsoft/Phi-3-mini-128k-instruct",
    "microsoft/Phi-3-small-128k-instruct",
    "microsoft/Phi-3-medium-128k-instruct",
    "microsoft/Phi-3.5-mini-instruct",
    "microsoft/phi-4",
    
    "meta-llama/Llama-3.2-3B-Instruct",
    "meta-llama/Llama-3.2-1B-Instruct",
    "meta-llama/Llama-3.1-8B-Instruct",
    
    "google/gemma-2-9b-it",
    "google/gemma-2-2b-it",

    "AI-MO/NuminaMath-7B-CoT",

    "internlm/internlm2_5-1_8b-chat",
    "internlm/internlm2_5-7b-chat",
    "internlm/internlm2_5-20b-chat",
    "internlm/internlm2-math-plus-1_8b",
    "internlm/internlm2-math-plus-20b",
    "internlm/internlm2-math-plus-7b",
    
    "deepseek-ai/deepseek-math-7b-instruct",
    "deepseek-ai/deepseek-math-7b-rl",
    
    "PowerInfer/SmallThinker-3B-Preview",

    "Skywork/Skywork-o1-Open-Llama-3.1-8B"
]
os.environ["CUDA_VISIBLE_DEVICES"] = "5"
print(len(model_list))

In [2]:
# First define all unique chat templates
CHAT_TEMPLATE_FORMATS = {
    "mistral_format": "<s>[INST] {system_message}\n\n{user_message}[/INST]",
    
    "qwen_format": "<|im_start|>system\n{system_message}<|im_end|>\n<|im_start|>user\n{user_message}<|im_end|>\n<|im_start|>assistant\n",
    
    "phi3mini_format": "<|system|>\n{system_message}<|end|>\n<|user|>\n{user_message}<|end|>\n<|assistant|>\n",
    
    "phi3small_format": "<|endoftext|><|system|>\n{system_message}<|end|>\n<|user|>\n{user_message}<|end|>\n<|assistant|>\n",
    
    "phi3medium_format": "<|user|>\n{user_message}<|end|>\n<|assistant|>\n",
    
    "phi4_format": "<|im_start|>system<|im_sep|>{system_message}<|im_end|><|im_start|>user<|im_sep|>{user_message}<|im_end|><|im_start|>assistant<|im_sep|>",
    
    "llama_format": "<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n{system_message}<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n{user_message}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n",
    
    "gemma_format": "<bos><start_of_turn>user\n{user_message}<end_of_turn>\n<start_of_turn>model\n",
    
    "numina_format": "### Problem: {user_message}\n### Solution: ",
    
    "internlm_format": "<s><|im_start|>system\n{system_message}<|im_end|>\n<|im_start|>user\n{user_message}<|im_end|>\n<|im_start|>assistant\n",
    
    "deepseek_format": "<｜begin▁of▁sentence｜>{system_message}\n\nUser: {user_message}\n\nAssistant:"
}

# Then map each model to its template format
MODEL_TO_TEMPLATE = {
    # Mistral Family
    "mistralai/Mistral-7B-Instruct-v0.3": CHAT_TEMPLATE_FORMATS["mistral_format"],
    "mistralai/Ministral-8B-Instruct-2410": CHAT_TEMPLATE_FORMATS["mistral_format"],
    "mistralai/Mistral-Nemo-Instruct-2407": CHAT_TEMPLATE_FORMATS["mistral_format"],
    "mistralai/Mistral-Small-Instruct-2409": CHAT_TEMPLATE_FORMATS["mistral_format"],
    "mistralai/Mathstral-7B-v0.1": CHAT_TEMPLATE_FORMATS["mistral_format"],
    
    # Qwen Family
    "Qwen/Qwen2.5-32B-Instruct": CHAT_TEMPLATE_FORMATS["qwen_format"],
    "Qwen/Qwen2.5-14B-Instruct": CHAT_TEMPLATE_FORMATS["qwen_format"],
    "Qwen/Qwen2.5-7B-Instruct": CHAT_TEMPLATE_FORMATS["qwen_format"],
    "Qwen/Qwen2.5-3B-Instruct": CHAT_TEMPLATE_FORMATS["qwen_format"],
    "Qwen/Qwen2.5-1.5B-Instruct": CHAT_TEMPLATE_FORMATS["qwen_format"],
    "Qwen/Qwen2.5-0.5B-Instruct": CHAT_TEMPLATE_FORMATS["qwen_format"],
    "Qwen/Qwen2.5-Math-7B-Instruct": CHAT_TEMPLATE_FORMATS["qwen_format"],
    "Qwen/Qwen2.5-Math-1.5B-Instruct": CHAT_TEMPLATE_FORMATS["qwen_format"],
    "Qwen/QwQ-32B-Preview": CHAT_TEMPLATE_FORMATS["qwen_format"],
    
    # Phi Family
    "microsoft/Phi-3-mini-128k-instruct": CHAT_TEMPLATE_FORMATS["phi3mini_format"],
    "microsoft/Phi-3.5-mini-instruct": CHAT_TEMPLATE_FORMATS["phi3mini_format"],
    "microsoft/Phi-3-small-128k-instruct": CHAT_TEMPLATE_FORMATS["phi3small_format"],
    "microsoft/Phi-3-medium-128k-instruct": CHAT_TEMPLATE_FORMATS["phi3medium_format"],
    "microsoft/phi-4": CHAT_TEMPLATE_FORMATS["phi4_format"],
    
    # Llama Family
    "meta-llama/Llama-3.2-3B-Instruct": CHAT_TEMPLATE_FORMATS["llama_format"],
    "meta-llama/Llama-3.2-1B-Instruct": CHAT_TEMPLATE_FORMATS["llama_format"],
    "meta-llama/Llama-3.1-8B-Instruct": CHAT_TEMPLATE_FORMATS["llama_format"],
    
    # Gemma Family
    "google/gemma-2-9b-it": CHAT_TEMPLATE_FORMATS["gemma_format"],
    "google/gemma-2-2b-it": CHAT_TEMPLATE_FORMATS["gemma_format"],
    
    # NuminaMath
    "AI-MO/NuminaMath-7B-CoT": CHAT_TEMPLATE_FORMATS["numina_format"],
    
    # InternLM Family
    "internlm/internlm2_5-1_8b-chat": CHAT_TEMPLATE_FORMATS["internlm_format"],
    "internlm/internlm2_5-7b-chat": CHAT_TEMPLATE_FORMATS["internlm_format"],
    "internlm/internlm2_5-20b-chat": CHAT_TEMPLATE_FORMATS["internlm_format"],
    "internlm/internlm2-math-plus-1_8b": CHAT_TEMPLATE_FORMATS["internlm_format"],
    "internlm/internlm2-math-plus-7b": CHAT_TEMPLATE_FORMATS["internlm_format"],
    "internlm/internlm2-math-plus-20b": CHAT_TEMPLATE_FORMATS["internlm_format"],
    
    # DeepSeek Math
    "deepseek-ai/deepseek-math-7b-instruct": CHAT_TEMPLATE_FORMATS["deepseek_format"],
    "deepseek-ai/deepseek-math-7b-rl": CHAT_TEMPLATE_FORMATS["deepseek_format"],
    
    # SmallThinker
    "PowerInfer/SmallThinker-3B-Preview": CHAT_TEMPLATE_FORMATS["qwen_format"],  # Uses same format as Qwen
    
    # Skywork
    "Skywork/Skywork-o1-Open-Llama-3.1-8B": CHAT_TEMPLATE_FORMATS["llama_format"]
}

In [ ]:
# question 1
with open("sbs_hard_q1.txt", "a") as f:
    for checkpoint in model_list:
        print("<<< checkpoint: ", checkpoint)
        f.write("<<< checkpoint: " + checkpoint + "\n")
        tokenizer = AutoTokenizer.from_pretrained(checkpoint, trust_remote_code=True)
        model = AutoModelForCausalLM.from_pretrained(
            checkpoint,
            trust_remote_code=True,
            device_map="cuda",
            torch_dtype="auto"
        )
        sbs_hard_prompt = """Please reason step by step, and put your final answer within \\boxed{{}}."""
        question = "What is $10.0000198\\cdot 5.9999985401\\cdot 6.9999852$ to the nearest whole number"
        if "gemma" in checkpoint:
            prompt = MODEL_TO_TEMPLATE[checkpoint].format(user_message=question)
        else:
            prompt = MODEL_TO_TEMPLATE[checkpoint].format(system_message=sbs_hard_prompt, user_message=question)
        inputs = tokenizer([prompt], return_tensors="pt").to(model.device)
        out = model.generate(**inputs, max_new_tokens=4096, do_sample=False)
        generated_ids = out[0][len(inputs.input_ids[0]):]
        response = tokenizer.decode(generated_ids, skip_special_tokens=True)
        print("\n<<< response:")
        print(response)
        print("-" * 100 + "\n")
        f.write(response + "\n")
        f.write("-" * 100 + "\n\n")
        del tokenizer
        del model

In [ ]:
# question 2
os.environ["CUDA_VISIBLE_DEVICES"] = "4"
with open("sbs_hard_q2.txt", "w") as f:
    for checkpoint in model_list:
        print("<<< checkpoint: ", checkpoint)
        f.write("<<< checkpoint: " + checkpoint + "\n")
        tokenizer = AutoTokenizer.from_pretrained(checkpoint, trust_remote_code=True)
        model = AutoModelForCausalLM.from_pretrained(
            checkpoint,
            trust_remote_code=True,
            device_map="cuda",
            torch_dtype="auto"
        )
        sbs_hard_prompt = """Please reason step by step, and put your final answer within \\boxed{{}}."""
        question = "Find the point of intersection of the line\n\\[\\frac{x - 2}{3} = \\frac{y + 1}{4} = \\frac{z - 2}{12}\\]and $x - y + z = 5.$"
        if "gemma" in checkpoint:
            prompt = MODEL_TO_TEMPLATE[checkpoint].format(user_message=question)
        else:
            prompt = MODEL_TO_TEMPLATE[checkpoint].format(system_message=sbs_hard_prompt, user_message=question)
        inputs = tokenizer([prompt], return_tensors="pt").to(model.device)
        out = model.generate(**inputs, max_new_tokens=4096, do_sample=False)
        generated_ids = out[0][len(inputs.input_ids[0]):]
        response = tokenizer.decode(generated_ids, skip_special_tokens=True)
        print("\n<<< response:")
        print(response)
        print("-" * 100 + "\n")
        f.write(response + "\n")
        f.write("-" * 100 + "\n\n")
        del tokenizer
        del model

In [ ]:
# question 3
os.environ["CUDA_VISIBLE_DEVICES"] = "5"
with open("sbs_hard_q3.txt", "w") as f:
    for checkpoint in model_list:
        print("<<< checkpoint: ", checkpoint)
        f.write("<<< checkpoint: " + checkpoint + "\n")
        tokenizer = AutoTokenizer.from_pretrained(checkpoint, trust_remote_code=True)
        model = AutoModelForCausalLM.from_pretrained(
            checkpoint,
            trust_remote_code=True,
            device_map="cuda",
            torch_dtype="auto"
        )
        sbs_hard_prompt = """Please reason step by step, and put your final answer within \\boxed{{}}."""
        question = "How many prime numbers less than 100 have a units digit of 3?"
        if "gemma" in checkpoint:
            prompt = MODEL_TO_TEMPLATE[checkpoint].format(user_message=question)
        else:
            prompt = MODEL_TO_TEMPLATE[checkpoint].format(system_message=sbs_hard_prompt, user_message=question)
        inputs = tokenizer([prompt], return_tensors="pt").to(model.device)
        out = model.generate(**inputs, max_new_tokens=4096, do_sample=False)
        generated_ids = out[0][len(inputs.input_ids[0]):]
        response = tokenizer.decode(generated_ids, skip_special_tokens=True)
        print("\n<<< response:")
        print(response)
        print("-" * 100 + "\n")
        f.write(response + "\n")
        f.write("-" * 100 + "\n\n")
        del tokenizer
        del model

In [ ]:

# step by step hard
sbs_hard_prompt = """Please reason step by step, and put your final answer within \\boxed{{}}."""

# step by step
sbs_prompt = """Please reason step by step, and put your final answer within \\boxed{{}} when done reasoning or early-stop keyword **Final Answer** appears.
"""

# coarse to fine
c2f_prompt = """Solve the task by following format:
**Coarse Reasoning**
Short analysis and an answer. Focus on efficiency and simplicity.

**Fine Reasoning**
Detailed analysis step by step and a refined answer. Focus on accuracy and correctness.

**Final Answer**
Your final answer within \\boxed{{}} when done reasoning or early-stop keyword **Final Answer** appears.
"""

# knowledge first
kf_prompt = """Solve the task by following format:
**Knowledge to Use**
List theorem and methods that are useful for solving the problem.

**Reasoning**
Step by step analysis using above knowledge.

**Final Answer**
Your final answer within \\boxed{{}} when done reasoning or early-stop keyword **Final Answer** appears.
"""

# answer and verification
aav_prompt = """Solve the task by following format:
**Quick Answer**
Give an initial answer based on intuition or quick calculation.

**Verification**
Give a revised answer through reasoning step by step to correct potential mistakes.

**Final Answer**
Your final answer within \\boxed{{}} when done reasoning or early-stop keyword **Final Answer** appears.
"""

In [ ]:
# check model chat template
# system_prompt = "Please reason step by step, and put your final answer within \\boxed{{}}."
# system_prompt = "Please reason step by step, and put your final answer within \\boxed{{}} when done reasoning or early-stop keyword **Final Answer** appears.
system_prompt = """Solve the task by following format:
**Knowledge to Use**
List theorem and methods that are useful for solving the problem.

**Reasoning**
Step by step analysis using above knowledge.

**Final Answer**
Your final answer within \\boxed{{}} when done reasoning or early-stop keyword **Final Answer** appears.
"""
# system_prompt = "Solve the task by following format:\n**Coarse Reasoning**\nShort analysis and an answer. Focus on efficiency and simplicity.\n\n**Fine Reasoning**\nDetailed analysis step by step and a refined answer. Focus on accuracy and correctness.\n\n**Final Answer** \nYour final answer within \\boxed{{}} when done reasoning or early-stop keyword **Final Answer** appears."

question = "What is $10.0000198\\cdot 5.9999985401\\cdot 6.9999852$ to the nearest whole number?" + system_prompt
messages = [{"role": "user", "content": question}]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
print(text)
print(type(text))
inputs = tokenizer([question], return_tensors="pt").to(model.device)
print(tokenizer.chat_template)
print(tokenizer.bos_token)
print(tokenizer.eos_token)
out = model.generate(**inputs, max_new_tokens=4096, do_sample=False)
print(tokenizer.decode(out[0]))
# prompt = question + "\nPlease reason step by step, and put your final answer within \\boxed{{}}." + "1. The robe takes 2 bolts of blue fiber.\n2. It takes half as much white fiber as blue fiber." + "\n\nFinal answer within \\boxed{{}}:\n"
# inputs = tokenizer([prompt], return_tensors="pt").to(model.device)
# prompt = question + "Solve the task through two types of reasoning:\n1. Coarse-Grained Reasoning: give quick analysis step by step and an answer. Focus on efficiency and simplicity.\n2. Fine-Grained Reasoning: give detailed analysis step by step and a refined answer. Foucus on accuracy and correctness.\nPut final answer within \\boxed{{}}.\n\nOutput format:\n**Coarse Reasoning**\n\n**Fine Reasoning**\n\n**Final Answer** within \\boxed{{}}"
# inputs = tokenizer([prompt], return_tensors="pt").to(model.device)
# "To solve this problem, we start by understanding that the robe requires 2 bolts of blue fiber. The problem states that the robe also requires half as much white fiber as blue fiber."

In [ ]:
input_template = "<|im_start|>system\n{system_prompt}<|im_end|>\n"
input_template += "<|im_start|>user\n{user_prompt}<|im_end|>\n"
input_template += "<|im_start|>assistant\n{assistant_prompt}\n"

step_by_step_hard = "Please reason step by step, and put your final answer within \\boxed{{}}."

step_by_step = "Please reason step by step, and put your final answer within \\boxed{{}} when done reasoning or early-stop keyword **Final Answer** appears."

coarse_to_fine = "Solve the task by following format:\n**Coarse Reasoning**\nShort analysis and an answer. Focus on efficiency and simplicity.\n\n**Fine Reasoning**\nDetailed analysis step by step and a refined answer. Focus on accuracy and correctness.\n\n**Final Answer** \nYour final answer within \\boxed{{}} when done reasoning or early-stop keyword **Final Answer** appears."


user_prompt = """What is $10.0000198\\cdot 5.9999985401\\cdot 6.9999852$ to the nearest whole number?"""

full_cot= "So I have this problem: calculate \\(10.0000198 \\times 5.9999985401 \\times 6.9999852\\) and round it to the nearest whole number. At first glance, it looks like a straightforward multiplication followed by rounding, but the numbers are pretty close to each other, which might make things interesting.\n\nFirst, I need to multiply these three numbers together. Since they're all decimals, multiplying them directly might be messy with so many decimal places. Maybe there's a smarter way to approach this without getting bogged down in too much calculation.\n\nLet me look at each number individually:\n\n- \\(10.0000198\\)\n\n- \\(5.9999985401\\)\n\n- \\(6.9999852\\)\n\nI notice that each of these numbers is very close to a certain integer value. For example:\n\n- \\(10.0000198\\) is almost \\(10\\)\n\n- \\(5.9999985401\\) is almost \\(6\\)\n\n- \\(6.9999852\\) is almost \\(7\\)\n\nThis makes me think that maybe these numbers are intended to represent integers plus some small fractions, and their product will simplify nicely.\n\nLet me try to express each number as the sum of its integer part and a small fractional part:\n\n- \\(10.0000198 = 10 + 0.0000198\\)\n\n- \\(5.9999985401 = 6 - 0.0000014599\\)\n\n- \\(6.9999852 = 7 - 0.0000148\\)\n\nWait, actually, let me double-check those fractional parts:\n\nFor \\(5.9999985401\\):\n\n\\(6 - 5.9999985401 = 0.0000014599\\)\n\nSimilarly for \\(6.9999852\\):\n\n\\(7 - 6.9999852 = 0.0000148\\)\n\nHmm, my initial assessment was slightly off. So, more accurately:\n\n- \\(10.0000198 = 10 + 0.0000198\\)\n\n- \\(5.9999985401 = 6 + 0.0000014599\\)\n\n- \\(6.9999852 = 7 - 0.0000148\\)\n\nNow, if I multiply these together:\n\n\\((10 + 0.0000198) \\times (6 + 0.0000014599) \\times (7 - 0.0000148)\\)\n\nMultiplying sums can be complicated, especially with such small fractional parts. Maybe there's another way to approximate this product.\n\nAlternatively, since the numbers are very close to integers, perhaps their product is also close to an integer, and I just need to find out which one it's closest to.\n\nLet me consider the product of the first two numbers:\n\n\\(10.0000198 \\times 5.9999985401\\)\n\nIf I treat these as approximately equal to 10 and 6, respectively, then:\n\n\\(10 \\times 6 = 60\\)\n\nBut since both numbers are slightly less than 10 and 6, respectively, their product should be slightly less than 60.\n\nSimilarly, multiplying by the third number, which is slightly more than 7, would adjust the product accordingly.\n\nMaybe I can use differentials to estimate the effect of these small deviations from the integers.\n\nLet me recall that for a function \\(f(x)\\), the differential \\(df \\approx f'(x) dx\\).\n\nIn this case, I can consider each number as a variable and see how changes in them affect the product.\n\nLet me denote:\n\n\\(a = 10.0000198\\)\n\n\\(b = 5.9999985401\\)\n\n\\(c = 6.9999852\\)\n\nThen, the product is \\(p = a \\times b \\times c\\)\n\nTaking natural logarithm on both sides:\n\n\\(\\ln p = \\ln a + \\ln b + \\ln c\\)\n\nNow, using Taylor expansion around \\(a_0, b_0, c_0\\) where \\(a_0 = 10\\), \\(b_0 = 6\\), \\(c_0 = 7\\):\n\n\\(\\ln p \\approx (\\ln a_0 + \\frac{a - a_0}{a_0} \\frac{d^2 \\ln a}{da^2}) + (\\ln b_0 + \\frac{b - b_0}{b_0} \\frac{d^2 \\ln b}{db^2}) + (\\ln c_0 + \\frac{c - c_0}{c_0} \\frac{d^2 \\ln c}{dc^2})\\)\n\nWait, this seems overly complicated. Maybe I should stick to linear approximation.\n\nLet me compute the partial derivatives of \\(\\ln p\\) with respect to \\(a\\), \\(b\\), and \\(c\\):\n\n\\(\\frac{\\partial \\ln p}{\\partial a} = \\frac{1}{a}\\)\n\n\\(\\frac{\\partial \\ln p}{\\partial b} = \\frac{1}{b}\\)\n\n\\(\\frac{\\partial \\ln p}{\\partial c} = \\frac{1}{c}\\)\n\nThen, the change in \\(\\ln p\\) due to small changes in \\(a\\), \\(b\\), and \\(c\\) is approximately:\n\n\\(\\delta \\ln p \\approx \\frac{1}{a}(a - a_0) + \\frac{1}{b}(b - b_0) + \\frac{1}{c}(c - c_0)\\)\n\nPlugging in the values:\n\n\\(a - a_0 = 10.0000198 - 10 = 0.0000198\\)\n\n\\(b - b_0 = 5.9999985401 - 6 = -0.0000014599\\)\n\n\\(c - c_0 = 6.9999852 - 7 = -0.0000148\\)\n\nSo,\n\n\\(\\delta \\ln p \\approx \\frac{1}{10}(0.0000198) + \\frac{1}{6}(-0.0000014599) + \\frac{1}{7}(-0.0000148)\\)\n\nCalculating each term:\n\n\\(\\frac{1}{10} \\times 0.0000198 = 0.00000198\\)\n\n\\(\\frac{1}{6} \\times (-0.0000014599) = -0.00000024331666...\\)\n\n\\(\\frac{1}{7} \\times (-0.0000148) = -0.0000021142857142857142...\\)\n\nAdding them up:\n\n\\(0.00000198 - 0.00000024331666... - 0.0000021142857142857142... = -0.0000003775923775923775...\\)\n\nTherefore,\n\n\\(\\ln p \\approx \\ln(10 \\times 6 \\times 7) + \\delta \\ln p\\)\n\nSince \\(10 \\times 6 \\times 7 = 420\\),\n\n\\(\\ln p \\approx \\ln 420 - 0.0000003775923775923775...\\)\n\nComputing \\(\\ln 420\\):\n\n\\(\\ln 420 \\approx 6.040527779779047\\)\n\nSo,\n\n\\(\\ln p \\approx 6.040527779779047 - 0.0000003775923775923775... \\approx 6.040527399999999...\\)\n\nExponentiating back to get \\(p\\):\n\n\\(p \\approx e^{6.040527399999999} \\approx 420 - 0.0000003775923775923775 \\times 604.0527779779047 \\approx 420 - 0.000232...\\)\n\nWait, that doesn't seem right. Let me check my calculations again.\n\nActually, \\(e^{-x}\\) is approximately \\(1 - x\\) for small \\(x\\), but here I have \\(e^{\\ln p}\\), which should give me \\(p\\). But I'm not sure about the exponentiation step.\n\nAlternatively, perhaps I should consider that the product is very close to 420, given that the adjustments are negligible.\n\nGiven that the adjustments are extremely small, the product should be very close to 420.\n\nTo confirm, let's consider the product of the three numbers directly:\n\n\\(10.0000198 \\times 5.9999985401 \\times 6.9999852\\)\n\nUsing a calculator or computing software, we can find the exact product.\n\nBut since I don't have access to that right now, I'll proceed with the assumption that the product is very close to 420.\n\nGiven that, rounding 420 to the nearest whole number is simply 420.\n\nHowever, to ensure accuracy, perhaps I should consider the possible range of the product.\n\nThe smallest possible product would occur if all three numbers are at their minimum values:\n\n\\(10.0000198 \\times 5.9999985401 \\times 6.9999852\\)\n\nAnd the largest possible product would be if they are at their maximum values, but since they are all less than their integer counterparts, the product should still be less than 420.\n\nWait, no. Actually, since each number is less than its integer counterpart, their product should be less than 420.\n\nWait, hold on:\n\n- \\(10.0000198 < 10\\)\n\n- \\(5.9999985401 < 6\\)\n\n- \\(6.9999852 > 7\\)\n\nWait, no, \\(6.9999852 < 7\\), because 6.9999852 is less than 7.\n\nWait, is that correct?\n\nLet me check:\n\n\\(6.9999852\\) is indeed less than 7, because subtracting any positive number from 7 gives a result greater than 6.9999852.\n\nSo, \\(6.9999852 < 7\\)\n\nTherefore, the product should be less than \\(10 \\times 6 \\times 7 = 420\\)\n\nBut wait, \\(10.0000198 > 10\\), \\(5.9999985401 < 6\\), and \\(6.9999852 < 7\\)\n\nSo, the product is greater than \\(10 \\times 6 \\times 7 = 420\\), but less than \\(11 \\times 5 \\times 7\\), which is \\(385\\), but that doesn't make sense because \\(10 \\times 6 \\times 7 = 420\\), and since each factor is less than its integer counterpart, the product should be less than 420.\n\nWait, no, \\(10.0000198 > 10\\), \\(5.9999985401 < 6\\), and \\(6.9999852 < 7\\)\n\nSo, the product is greater than \\(10 \\times 6 \\times 7 = 420\\), but less than \\(11 \\times 5 \\times 7 = 385\\), which is impossible because 420 is already greater than 385.\n\nWait, no, \\(11 \\times 5 \\times 7 = 385\\), which is less than 420, but our product is greater than 420 because one factor is greater than 10 and the others are less than their integer values.\n\nWait, no, \\(10.0000198 > 10\\), \\(5.9999985401 < 6\\), and \\(6.9999852 < 7\\)\n\nSo, the product is greater than \\(10 \\times 6 \\times 7 = 420\\), but less than \\(11 \\times 5 \\times 7 = 385\\), which is impossible.\n\nWait, no, \\(11 \\times 5 \\times 7 = 385\\), which is less than 420, but our product has one factor greater than 10 and the others less than their integer values, so it should be between 420 and 385, which is impossible.\n\nWait, perhaps I made a mistake in assessing the ranges.\n\nLet me re-express the numbers:\n\n- \\(10.0000198 = 10 + 0.0000198\\)\n\n- \\(5.9999985401 = 6 - 0.0000014599\\)\n\n- \\(6.9999852 = 7 - 0.0000148\\)\n\nSo, the product is:\n\n\\((10 + 0.0000198) \\times (6 - 0.0000014599) \\times (7 - 0.0000148)\\)\n\nExpanding this:\n\nFirst, multiply the first two terms:\n\n\\((10 + 0.0000198)(6 - 0.0000014599) = 10 \\times 6 + 10 \\times (-0.0000014599) + 0.0000198 \\times 6 + 0.0000198 \\times (-0.0000014599)\\)\n\nWhich simplifies to:\n\n\\(60 - 0.000014599 + 0.0001188 - 0.0000000003\\)\n\nApproximately:\n\n\\(60 - 0.000014599 + 0.0001188 - 0.0000000003 \\approx 60.0001042007\\)\n\nNow, multiply this by the third term:\n\n\\(60.0001042007 \\times (7 - 0.0000148) = 60.0001042007 \\times 7 - 60.0001042007 \\times 0.0000148\\)\n\nCalculating each part:\n\n\\(60.0001042007 \\times 7 = 420.0007294049\\)\n\n\\(60.0001042007 \\times 0.0000148 = 0.0008712007\\)\n\nSubtracting:\n\n\\(420.0007294049 - 0.0008712007 \\approx 420.0007294049 - 0.0008712007 = 419.9998582042\\)\n\nSo, the product is approximately 419.9998582042\n\nRounding this to the nearest whole number:\n\nLooking at the decimal part, 0.999998582042, which is very close to 1, so rounding up to 420.\n\nTherefore, the nearest whole number is 420.\n\n**Final Answer**\n\n\\[ \\boxed{420} \\]<|im_end|>"

ratio = 0.5
assistant_cot = ""
if ratio > 0.0:
    assistant_cot = full_cot[:int(len(full_cot) * ratio)]
    assistant_cot += "\n\n**Final Answer** \n"

prompt = input_template.format(system_prompt=coarse_to_fine, user_prompt=user_prompt, assistant_prompt=assistant_cot)

In [ ]:
print("formatted input prompt:")
print("-" * 100)
print(prompt)

model_inputs = tokenizer([prompt], return_tensors="pt").to("cuda")

generated_ids = model.generate(
    **model_inputs,
    do_sample=False,
    max_new_tokens=4096 if ratio < 0.0 else 25
)

generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
print("response:")
print(response)
